In [61]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from nltk import tokenize

d_set = pd.read_csv(r"C:\Users\pc\Desktop\spam.csv", encoding="ISO-8859-1")
label_encoder = LabelEncoder()
d_set['v1'] = label_encoder.fit_transform(d_set['v1'])
#removing null values
d_set = d_set.dropna(axis=1, how='any') 

# lowercasing
d_set['v2'] = d_set['v2'].str.lower()

# removing stop words and stemmers 
# init 
stop_words = set(stopwords.words('english'))

def remove_swp(text):
    # remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    # remove stopwords 
    return " ".join([word for word in text.split() if word.lower() not in stop_words])
d_set['v2'] = d_set['v2'].apply(remove_swp)

d_set

,v1,v2
0,0,go jurong point crazy available bugis n great ...
1,0,ok lar joking wif u oni
2,1,free entry 2 wkly comp win fa cup final tkts 2...
3,0,u dun say early hor u c already say
4,0,nah dont think goes usf lives around though
...,...,...
5567,1,2nd time tried 2 contact u u å£750 pound prize...
5568,0,ì b going esplanade fr home
5569,0,pity mood soany suggestions
5570,0,guy bitching acted like id interested buying s...


In [ ]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
c = CountVectorizer()
tfid = TfidfVectorizer(max_features = 3000)
#removing pct and numbers
def remove_np(text):
   return re.sub(r'\d+', '', text)
d_set['v2'] = d_set['v2'].apply(remove_np)

# Tokenize
def tokenize_text(text):
    return word_tokenize(text)
d_set['v3'] = d_set['v2'].apply(tokenize_text)

d_set 

In [67]:
# single string
d_set['v3'] = d_set['v3'].apply(lambda x: ' '.join(x))
X = tfid.fit_transform(d_set['v3']).toarray()
X_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
import joblib
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X_array, d_set['v1'], test_size=0.2, random_state = 2)

# bayes
model = BernoulliNB()
model.fit(X_train, y_train)

joblib.dump(model, 'spam_detection.joblib')
joblib.dump(c, 'spam_count.joblib')
y_pred = model.predict(X_test)
score = accuracy_score(y_test, y_pred)
display('X-Train :', X_train.shape)
display('X-Test :',X_test.shape)
display('Y-Train :',y_train.shape)
display('X-Test :',y_test.shape)

'X-Train :'

(4457, 8460)

'X-Test :'

(1115, 8460)

'Y-Train :'

(4457,)

'X-Test :'

(1115,)